<a href="https://colab.research.google.com/github/HakanOznam/Makine-Ogrenmesi-ile-Satranc-Oyun-Sonu-Tahmini/blob/main/Chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kütüphanelerin import edilmesi ve veri kümesinin dahil edilmesi

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay ,classification_report

data=pd.read_excel("games.xlsx")

Analizin doğruluğunu arttırmak için hamle sayısı 10'dan küçük olan maçları veri kümesinden silme işlemi

In [13]:
data=data.drop(data.loc[data['turns']<=10].index)
data=data.reset_index(drop=True)

Eksik veri kontrolü

In [ ]:
data.isnull().sum()

Eksik verilerin IterativeImputer metodu ile işlenmesi 

In [15]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
testmissing = data.iloc[:9000,2:4]
trainmissing = data.iloc[9000:,2:4]
imp = IterativeImputer(random_state=0)
imp.fit(trainmissing)
data.iloc[:9000,2:4] = np.round(imp.transform(testmissing))

Kategorik verilerin sayısal verilere dönüştürme işlemi

In [16]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

#rated özniteliği için 
data.iloc[:,1] = le.fit_transform(data.iloc[:,1])

#victory status özniteliği için
ohe = preprocessing.OneHotEncoder()
vicstatus = data.iloc[:,4].to_numpy()
vicstatus = vicstatus.reshape(-1,1)
ohe_vicstatus = ohe.fit_transform(vicstatus).toarray()
dfvicstatus=pd.DataFrame(data=ohe_vicstatus,columns=["draw","mate","outoftime","resign"])

Oyun açılış(opening_eco) özniteliğinin sayısal verilere dönüştürme işlemi

In [ ]:
eco = data['opening_eco']
for i in range(len(eco)):  
    eco[i]=eco[i].replace("A","1")
    eco[i]=eco[i].replace("B","2")
    eco[i]=eco[i].replace("C","3")
    eco[i]=eco[i].replace("D","4")
    eco[i]=eco[i].replace("E","5")

eco=pd.to_numeric(eco)

Verilerin birleştirilmesi, etiket veri kolonunun belirlenmesi

In [18]:
X = pd.concat([data["rated"],data["game_time"],data["turns"],dfvicstatus,data["black_rating"],data["white_rating"]],axis=1)

y = data["winner"]
y = le.fit_transform(y)
classes =["black","draw","white"]

Verilerin standardize edilmesi


In [19]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_sc = sc.fit_transform(X)

KFold algoritması kullanılarak modellerin sonuçlarını yazdıran fonksiyon tanımlanması

In [45]:
def output(est,X,color):
    for model in est:
        accu = cross_val_score(estimator = model, X = X, y=y,cv=5)
        y_pred = cross_val_predict(model,X,y,cv=5)
        conf_matrix = confusion_matrix(y, y_pred)        
        """görselleştirme"""
        #disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,display_labels=["black","draw","white"])
        #disp.plot(cmap=color)
        #disp.ax_.set_title(model)
        print("-------- ",model," report -------- \n")
        print(classification_report(y,y_pred,target_names=classes))
        print("\n")

Modellerin import edilmesi

In [21]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0)

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()


GridSearchCV algoritması ile Random Forest Classifier

In [25]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

parameters = [{'n_estimators': [50,100,150,200], 'criterion':['gini',"entropy"]}]
grids = GridSearchCV(estimator = rfc, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grids.fit(X_sc, y)
best_accuracy , best_parameter = grids.best_score_ , grids.best_params_

In [41]:
grid_rfc = RandomForestClassifier(n_estimators=150,criterion="entropy")

GridSearchCV algoritması ile KNN Classifier

In [33]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

parameters = [{'n_neighbors': [3,5,7,9,11,13,15,17,19,21], 'weights':['uniform',"distance"],"metric" : ["euclidean","manhattan","minkowski"]}]
grids = GridSearchCV(estimator = knn, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grids.fit(X_sc, y)
best_accuracy , best_parameter = grids.best_score_ , grids.best_params_


{'metric': 'manhattan', 'n_neighbors': 21, 'weights': 'distance'}

In [42]:
grid_knn = KNeighborsClassifier(n_neighbors=21,weights="distance",metric="manhattan")

GridSearchCV algoritması ile Support Vector Classifier

In [ ]:
from sklearn.svm import SVC
svm = SVC()

parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel':['linear']}, {'C': [0.25, 0.5, 0.75, 1], 'kernel':['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5]}]
grids = GridSearchCV(estimator = svm, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grids.fit(X_sc, y)
best_accuracy , best_parameter = grids.best_score_ , grids.best_params_


In [43]:
grid_svm = SVC(C=0.75,kernel="linear")

Parametre kullanılmayan modellerin sonuçları

In [ ]:
est = [logreg,clf,rfc,knn,gnb]
output(est,X_sc,"pink")

GridSearchCV uygulanan üç modelin sonuçları

In [ ]:
grid_est = [grid_rfc,grid_knn,grid_svm]
output(grid_est,X_sc,"hot")


Boyut azaltma tekniği olan LDA algoritması 

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 2)
X_lda = lda.fit_transform(X_sc, y)

LDA ve GridSearchCV uygulanan modellerin sonuçları

In [ ]:
grid_lda_est = [logreg,clf,grid_rfc,grid_knn,gnb]
output(grid_lda_est,X_lda,"bone")